#  Astrocyte Neural Network with Dynamic Modulation

In [2]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from icecream import ic


In [3]:
data = pd.read_csv('train.csv')
data = np.array(data)
m, n = data.shape
np.random.seed(0)
# np.random.shuffle(data) # shuffle before splitting into dev and training sets

data_dev = data[0:1000].T # make sure each column is an data piece
Y_dev = data_dev[0]
X_dev = data_dev[1:n]
X_dev = X_dev / 255.

data_train = data[1000:m].T
Y_train = data_train[0]
X_train = data_train[1:n]
X_train = X_train / 255.
_,m_train = X_train.shape

In [7]:
# data = pd.read_csv('planar_flower.csv')[['y', 'x1', 'x2']]
# data = np.array(data)
# m, n = data.shape
# np.random.seed(0)
# # np.random.shuffle(data) # shuffle before splitting into dev and training sets

# data_dev = data[0:1000].T # make sure each column is an data piece
# Y_dev = data_dev[0]
# X_dev = data_dev[1:n]
# X_dev = X_dev / 255.

# data_train = data[1000:m].T
# Y_train = data_train[0]
# X_train = data_train[1:n]
# X_train = X_train / 255.
# _,m_train = X_train.shape

In [ ]:
from ann import AstrocyteNetwork
sizes = [784, 10, 10]  # Example network architecture

astrocyte_net = AstrocyteNetwork(sizes, astrocyte_density=1.0, initial_threshold=0.5, initial_effect=0.1)

# # Train the network
# # astrocyte_net.astrocyte_thresholds
astrocyte_net.gradient_descent(X_train, Y_train, alpha=0.015, iterations=1000)

# # Visualize astrocyte parameters
# # astrocyte_net.plot_astrocyte_params()

In [ ]:
from dynamic_ann import AstrocyteNetwork
sizes = [784, 10, 10]  # Example network architecture

astrocyte_net = AstrocyteNetwork(sizes, astrocyte_density=1.0, initial_threshold=0.5, initial_effect=0.1)

# astrocyte_net.astrocyte_thresholds[0][0][0]
# astrocyte_net.forward_prop(X_train[:, 0])
# # Train the network

astrocyte_net.gradient_descent(X_train, Y_train, alpha=0.015, iterations=1000)

# # Visualize astrocyte parameters
# # astrocyte_net.plot_astrocyte_params()

In [1]:
np.argmax(astrocyte_net.forward_prop(X_dev[:, 49])), Y_dev[49]

NameError: name 'np' is not defined